In [24]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    print(filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    
    
    #find tempo 
    tempo = midi_data.estimate_tempo()
    print("Tempo: " + str(tempo))
    f.write(str(tempo) + "\n")
    
    segment_duration = 15
    
    #in each segment in sliding window write the notes to a file
    for i in range(0, int(song_duration)-segment_duration):
        for note in midi_data.instruments[0].notes:
            if note.start > i and note.end < i + segment_duration:
                f.write(str(note.pitch) + " ")
        f.write("\n")
        
    f.close()
    


All_Out_Of_Love.1.mid
Tempo: 213.2872691017864
Angel_Eyes.mid
Tempo: 177.98377752028006
A_Love_So_Beautiful.mid
Tempo: 148.90908346363713
Cant_Touch_This.mid
Tempo: 258.3911272900179
Crawling.mid
Tempo: 207.4163045498015
Dancing_Machine.mid
Tempo: 209.25505541449166
Dancing_Queen.mid
Tempo: 210.25434498609175
Danza_Ungherese_n._8_in_la_minore-Presto.mid
Tempo: 224.7252824363204
Everybody_Backstreets_Back_.1.mid
Tempo: 205.3341258303746
For_Your_Love.mid
Tempo: 194.7963113011656
Ill_Be_There_For_You.mid
Tempo: 213.94133967876826
Its_OK.mid
Tempo: 234.08229711785214
Joan_of_Arc_Maid_of_Orleans_.mid
Tempo: 220.87110005189152
Never_Gonna_Give_You_Up.1.mid
Tempo: 235.89881499508306
November_Rain.1.mid
Tempo: 190.72669016087372
Step_By_Step.mid
Tempo: 239.2369352021855
ThankYou.mid
Tempo: 160.0
The_Real_Slim_Shady.mid
Tempo: 210.00315004725013
Words.mid
Tempo: 172.5235562880708
You_Gotta_Fight_for_Your_Right.1.mid
Tempo: 249.26854195146876


In [19]:
import librosa 

x,sr = librosa.load('input.wav')
onset_env = librosa.onset.onset_strength(x, sr=sr)
tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)
print(tempo)

TypeError: onset_strength() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

In [25]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    
    print("File: " + filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    print("Song duration: " + str(song_duration))
    
    #find the number of notes in the song
    print("Number of notes: " + str(len(midi_data.instruments[0].notes)))
    
    segment_duration = 30
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    
    #in each segment in sliding window write the notes to a file
    
    # for i in range(0, int(song_duration)-segment_duration):
    #     prev = 1
    #     for j in range(prev, len(midi_data.instruments[0].notes)):
    #         if midi_data.instruments[0].notes[i].start > i and midi_data.instruments[0].notes[i].end < i+segment_duration:
    #             #relative pitches should be written
    #             f.write(str(abs(midi_data.instruments[0].notes[i].pitch - midi_data.instruments[0].notes[i-1].pitch)) + " ")
    #         f.write("\n")
    #     else:
    #         prev = j
    #         continue
        
    # f.close()
    
    for i in range(0, int(song_duration) - segment_duration):
        prev = 1
        for j in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[j]
            if note.start > i and note.end < i + segment_duration:
                # relative pitches should be written
                f.write(str(abs(note.pitch - midi_data.instruments[0].notes[j - 1].pitch)) + " ")
        f.write("\n")
        prev = j + 1

    f.close()

File: All_Out_Of_Love.1.mid
Song duration: 234.50284799999997
Number of notes: 470
File: Angel_Eyes.mid
Song duration: 219.0
Number of notes: 944
File: A_Love_So_Beautiful.mid
Song duration: 247.61906
Number of notes: 39
File: Cant_Touch_This.mid
Song duration: 261.04407999999995
Number of notes: 636
File: Crawling.mid
Song duration: 200.76920399999997
Number of notes: 126
File: Dancing_Machine.mid
Song duration: 91.03441600000001
Number of notes: 1928
File: Dancing_Queen.mid
Song duration: 219.99978000000002
Number of notes: 276
File: Danza_Ungherese_n._8_in_la_minore-Presto.mid
Song duration: 152.76302083333331
Number of notes: 304
File: Everybody_Backstreets_Back_.1.mid
Song duration: 231.13974646614582
Number of notes: 3840
File: For_Your_Love.mid
Song duration: 311.0017
Number of notes: 2578
File: Ill_Be_There_For_You.mid
Song duration: 182.21025299999997
Number of notes: 357
File: Its_OK.mid
Song duration: 202.42266171874996
Number of notes: 2672
File: Joan_of_Arc_Maid_of_Orleans

In [26]:
from fastdtw import fastdtw

def get_dtw_distance(input_pitches, segment_pitches):
    distance, path = fastdtw(input_pitches, segment_pitches)
    
    return distance

In [27]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for i in range(1, len(input_midi_data.instruments[0].notes)):
    input_pitches.append(abs(input_midi_data.instruments[0].notes[i].pitch - input_midi_data.instruments[0].notes[i-1].pitch))

# for note in input_midi_data.instruments[0].notes:
#     input_pitches.append(note.pitch)
    
    
print("Input pitches: " + str(input_pitches))

top_5 = []

for filename in os.listdir("index_dataset"):
    smallest = 10000000
    best_segment = None
    f = open('index_dataset/' + filename, 'r')
    content = f.read()
    notes = content.split("\n")
    
    for note in notes:
        if note == '':
            continue
        #convert string to array of integers
        
        note = note.split(" ")
        #remove last element
        note.pop()
        note = list(map(int, note))
        distance = get_dtw_distance(input_pitches, note)
        if distance < smallest:
            smallest = distance
            best_segment = note
    
    print("File: " + filename)
    print("Distance: " + str(smallest))
    print("Segment: " + str(best_segment))        
    #top_5.append({"filename": filename, "distance": smallest, "segment": best_segment})
    
    
    f.close()
#top_5.sort(key=lambda x: x["distance"])
# for i in range(0, 5):
#     print("File: " + top_5[i]["filename"])
#     print("Distance: " + str(top_5[i]["distance"]))
#     print("Segment: " + str(top_5[i]["segment"]))
#     print("\n")
    
        

Input pitches: [0, 0, 0, 0, 1, 2, 1, 2, 4, 0, 0, 1, 2, 1, 3, 1, 3, 8, 7, 8, 7, 1, 0, 1, 1, 2]
File: All_Out_Of_Love.1.mid.txt
Distance: 28.0
Segment: [0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]
File: Angel_Eyes.mid.txt
Distance: 462.0
Segment: [2, 27, 18, 2, 19, 20, 20, 10, 2, 1, 4, 14, 8, 1, 12, 19, 21, 9, 4, 6, 15, 6, 1, 21, 15, 4, 0, 20, 17, 17, 5, 7, 13, 16, 5, 3, 19, 7, 9, 6, 3, 14, 17, 3, 15, 10, 1, 4, 2, 16, 19, 10, 10, 20, 16, 2, 9, 9, 2, 10, 4, 5, 3, 15, 1, 7, 2, 2, 9, 4, 7, 0, 4, 5, 0, 10, 5, 3, 9, 1, 6, 1]
File: A_Love_So_Beautiful.mid.txt
Distance: 25.0
Segment: [2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 5, 2, 1, 0, 1]
File: Cant_Touch_This.mid.txt
Distance: 108.0
Segment: [0, 5, 2, 2, 2, 1, 2, 5, 3, 4, 2, 5, 2, 1, 2, 2, 4, 4, 2, 5, 2, 1, 2, 5, 3, 4, 2, 5, 2, 1, 2, 2, 4, 4, 2, 5, 2, 1, 2, 5, 3, 4, 2, 5, 2, 1, 2, 2, 4, 4, 2, 5, 2, 1, 2, 5, 3, 4, 2, 5, 2, 1, 2, 2, 4, 4]
File: Crawling.mid.txt
Distance: 33.0
Segment: [2, 1, 1, 7, 3, 2, 2, 1, 1, 3, 10, 3, 2, 2, 1, 1]
File: Dancing_Machine.mid.txt

In [20]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    
)

print(es.ping())


True


In [21]:
#create index
mapping = {
    "mappings": {
        "properties": {
            'song': { 'type': 'text' },
            'notes': { 'type': 'text'},
            'tempo': { 'type': 'float'},
            'relative_pitches': { 'type': 'text'}
        }
    }
}

if(es.indices.exists(index="midi")):
    es.indices.delete(index="midi")

es.indices.create(index="midi", body=mapping, ignore=400)

C:\Users\soumil\AppData\Local\Temp\ipykernel_4432\2734717090.py:16: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index="midi", body=mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'midi'})

In [ ]:
for filename in os.listdir("index_dataset"):
    f = open("index_dataset/" + filename, "r")
    content = f.read()
    
    #each line should be a document
    line = content.split("\n")
    for i in range(0, len(line)):
        if (line[i]== ""):
            continue
        doc = {
            "song": filename,
            "notes": line[i]
        }
        es.index(index="midi", body=doc)
    
    print(filename + " indexed")
    
    f.close()

In [ ]:
from fastdtw import fastdtw

def get_dtw_distance(input_pitches, segment_pitches):
    distance, path = fastdtw(input_pitches, segment_pitches)
    
    return distance

In [ ]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
print(input_midi_data.estimate_tempo())

In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)

query = {
        'query': {
            'function_score': {
                'functions': [
                    {
                        'script_score': {
                            'script': {
                                'source': 'fastdtw(params.query_pitches, doc["notes"])',
                                'params': {
                                    'query_pitches': input_pitches
                                }
                            }
                        }
                    }
                ],
                'score_mode': 'avg'
            }
        }
}

result = es.search(index='midi', body=query)
print(result['hits']['hits'][0:3]['_source']['song']) 



In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)


dtw_script = {
        "script": {
            "source": "1.0 / (1.0 + get_dtw_distance(params.query_sequence, doc['features'].notes))",
            "lang": "painless",
        },
        "params": {"query_sequence": input_pitches},
    }

query = {
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "script_score": dtw_script
            }
        }
    }


results = es.search(index='midi', body=query)
print(results['hits']['hits'][:3])
